In [8]:
import pandas as pd
import pymongo
import numpy as np

In [50]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
mrk_ref = pd.read_table("../data/MGI/MRK_Reference.rpt",header=None,names=["mgi_marker_acc_id",
                                                                           "marker_symbol",
                                                                           "marker_name",
                                                                           "marker_syns",
                                                                           "pmid"])

In [33]:
ref_data = {"acc":[],"pmid":[]}

In [34]:
for tup in mrk_ref[["mgi_marker_acc_id","pmid"]].itertuples():
    pmids = [int(x) for x in tup.pmid.split("|")]
    ref_data["acc"]+=[tup.mgi_marker_acc_id]*len(pmids)
    ref_data["pmid"]+=pmids
#     print([tup.mgi_marker_acc_id]*len(pmids),pmids)

In [28]:
mrk_gene = pd.read_table("../data/MGI/MGI_EntrezGene.rpt",header=None,names=["mgi_marker_acc_id",
                                                                             "marker_symbol",
                                                                             "status",
                                                                             "marker_name",
                                                                             "cm_pos",
                                                                             "chromosome",
                                                                             "type",
                                                                             "sec_acc",
                                                                             "gene_id",
                                                                             "syns",
                                                                             "feature_type",
                                                                             "genome_coord_start",
                                                                             "genome_coord_end",
                                                                             "strand",
                                                                             "biotype"])

In [41]:
gene_pmid = mrk_gene[mrk_gene.gene_id.isnull()==False][["mgi_marker_acc_id",
                                            "gene_id"]].merge(pd.DataFrame(ref_data),
                                                              how="inner",
                                                              left_on="mgi_marker_acc_id",
                                                              right_on="acc"
                                                             )

In [51]:
to_insert = []
for i,row in gene_pmid[["gene_id","acc","pmid"]].iterrows():
    to_insert.append(dict(row))
    if i%25000==0:
        db.mgi.insert_many(to_insert)
        to_insert=[]
db.mgi.insert_many(to_insert)